<a href="https://colab.research.google.com/github/BenBronselaer/CMIP6/blob/master/CMIP6_average__temperature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install needed packages
!pip install xarray
!pip install zarr
!pip install gcsfs
!pip install cftime
!apt-get -qq install python-cartopy python3-cartopy
!git clone https://github.com/smartlixx/WetBulb/
!git clone https://github.com/smartlixx/WetBulb/

     |████████████████████████████████| 140 kB 8.6 MB/s 
     |████████████████████████████████| 6.1 MB 15.6 MB/s 
  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5049 sha256=9ef20ed0a2460440687e83718bcc3777c0e2421c72c4bea105c3a40c41826ce4
  Stored in directory: /root/.cache/pip/wheels/12/1c/38/0def51e15add93bff3f4bf9c248b94db0839b980b8535e72a0
Successfully built asciitree
     |████████████████████████████████| 1.3 MB 11.3 MB/s 
     |████████████████████████████████| 118 kB 52.0 MB/s 
     |████████████████████████████████| 294 kB 44.5 MB/s 
     |████████████████████████████████| 142 kB 54.1 MB/s 
Selecting previously unselected package python-pkg-resources.
(Reading database ... 160837 files and directories currently installed.)
Preparing to unpack .../00-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-pyshp.
Preparing to unpack .../01-python-pyshp_1.2.12+ds-1_all.de

In [2]:
# load packages
from matplotlib import pyplot as plt
import math
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs
#import cartopy.crs as ccrs
from scipy.signal import find_peaks
import scipy.stats as stats
import warnings
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from google.colab import files

import cftime
from datetime import datetime
xr.set_options(display_style='html')
plt.rcParams['figure.figsize'] = 12, 6
from WetBulb.WetBulb import WetBulb

In [3]:
# set parameters for the analysis
n_ens_members=15         # number of models used (each will use one ensmeble member), if this is more than the number of models available, the code will just use the number available
sig_level=1.0           # significant level for statistical analysis, this number is multiplied with the standard deviation to show the confidence interval in the mean, in the figures and the downloaded data spreadsheet

# load CMIP5 databases for google cloud
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')


# set model queries
# guide for names here:   https://docs.google.com/document/d/1yUx6jr9EdedCOLd--CPdTfGDwEwzPpCF6p1jRmqx-0Q 
# firstly set queries for both ensemble
# the format of single and double quotation marks is important, so make sure these are kept 
variables={'Average temperature':"'tas'" ,   'Daily max temperature':"'tasmax'" , 'Precipitation':"'pr'" ,
           'Relative humidity':"'hurs'" ,'Wind speed':"'sfcWind'" ,'Sea surface height':"'zos'" ,}
variable =variables['Average temperature']

resolution = "'Amon'" # this sets the time resolution of the data, see the google doc above
variant="'r1i1p1f1'" # this selects one particular mensembke member for each model




# then, set the historical ensemble details
# Don't need to change these
activity_1="'CMIP'"
scenario_1 = "'historical'"
date_1_start ="1951-01-16"
date_1_end = "2014-12-15"

# lastly, set the future ensemble details
activity_2="'ScenarioMIP'"
scenario_2 = "'ssp370'" # this is the future sclimate change scenario
date_2_start ="2015-01-16"
date_2_end = "2100-12-15"


# set coordinates for location to be analyzed
# use Tangguh as example in this case
xc=10
yc=52

# difference factor, set this to 273.15 for temperature (to use celsius instead of Kelvin)
# but keep as zero for other variables
diff_fac=273.15

# set what rolling mean to use for output
# default is 20 years
rolling_window=20


In [4]:
# see if data is available for choice of scenario, time resolution and variable. If there is no data available, the output here will be empty
df_hist = df.query("activity_id== " + activity_1 +" & table_id == "+resolution+" & variable_id == "+variable+" & experiment_id == "+scenario_1)
df_hist

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
12789,CMIP,NOAA-GFDL,GFDL-CM4,historical,r1i1p1f1,Amon,tas,gr1,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/histo...,NaN,20180701
14271,CMIP,NOAA-GFDL,GFDL-ESM4,historical,r3i1p1f1,Amon,tas,gr1,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/hist...,NaN,20180701
14302,CMIP,NOAA-GFDL,GFDL-ESM4,historical,r2i1p1f1,Amon,tas,gr1,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/hist...,NaN,20180701
22214,CMIP,IPSL,IPSL-CM6A-LR,historical,r1i1p1f1,Amon,tas,gr,gs://cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/histor...,NaN,20180803
22715,CMIP,IPSL,IPSL-CM6A-LR,historical,r17i1p1f1,Amon,tas,gr,gs://cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/histor...,NaN,20180803
...,...,...,...,...,...,...,...,...,...,...,...
506506,CMIP,MIROC,MIROC-ES2L,historical,r28i1p1f2,Amon,tas,gn,gs://cmip6/CMIP6/CMIP/MIROC/MIROC-ES2L/histori...,NaN,20210317
506614,CMIP,MIROC,MIROC-ES2L,historical,r15i1p1f2,Amon,tas,gn,gs://cmip6/CMIP6/CMIP/MIROC/MIROC-ES2L/histori...,NaN,20210317
508294,CMIP,AS-RCEC,TaiESM1,historical,r2i1p1f1,Amon,tas,gn,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/historic...,NaN,20210416
508312,CMIP,NIMS-KMA,UKESM1-0-LL,historical,r15i1p1f2,Amon,tas,gn,gs://cmip6/CMIP6/CMIP/NIMS-KMA/UKESM1-0-LL/his...,NaN,20210426


In [5]:
# data base queries
# historical ensemble query
df_hist = df.query("activity_id== " + activity_1 + " & member_id == "+variant+" & table_id == "+resolution+" & variable_id == "+variable+" & experiment_id == "+scenario_1)

# future ensemble query
df_ssp = df.query("activity_id== " + activity_2 + " & member_id == "+variant+"  & table_id == "+resolution+" & variable_id == "+variable+" & experiment_id == "+scenario_2)

# create list of available models
models = list(set(df_ssp['source_id']).intersection(df_hist['source_id']))


# If there are models that give errors during the loading later on, remove them here and run again
models.remove('MPI-ESM1-2-HR')


print('Models used:')
models

Models used:


['EC-Earth3',
 'IPSL-CM5A2-INCA',
 'FGOALS-f3-L',
 'ACCESS-CM2',
 'MPI-ESM-1-2-HAM',
 'GFDL-ESM4',
 'TaiESM1',
 'MPI-ESM1-2-LR',
 'INM-CM4-8',
 'CAMS-CSM1-0',
 'INM-CM5-0',
 'MIROC6',
 'NorESM2-LM',
 'CMCC-ESM2',
 'EC-Earth3-Veg-LR',
 'CAS-ESM2-0',
 'NorESM2-MM',
 'EC-Earth3-AerChem',
 'IITM-ESM',
 'ACCESS-ESM1-5',
 'CMCC-CM2-SR5',
 'BCC-CSM2-MR',
 'KACE-1-0-G',
 'CESM2-WACCM',
 'EC-Earth3-Veg',
 'AWI-CM-1-1-MR',
 'MRI-ESM2-0',
 'FGOALS-g3',
 'CanESM5',
 'IPSL-CM6A-LR']

In [6]:
## DEFINE FUNCTIONS FOR LATER USE


def standardize_dims(ds):
    # function to standardize dimension names in cmip models
    # dimension names get mapped to the keys in the rdic dictionary
    # ds = input array
    # code adapted from Julius Buseckes' CMIP6 pre-processing package
    rdic = {
        "lon": ["x", "i", "nlon", "lon"],
        "lat": ["y", "j", "nlat", "lat"],
        "lev": ["lev", "depth", "olevel", "zlev", "olev"],
        "time": ["time","t"],
        }
    for di in rdic.keys():
        if di not in ds.coords:
            for wrong in rdic[di]:
                if wrong in ds.coords:
                    ds=ds.rename({wrong: di})
    return ds


def plot_timeseries(input_arr,model_list,title1,title_2,axis1):
  config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
    'height': 800,
    'width': 1000,
    'scale': 2 # Multiply title/legend/axis/canvas sizes by this factor
    }
  }
  fig = make_subplots(rows=2, cols=1,x_title='year',
                    subplot_titles=(title1,  '% change since 1990-2010'),vertical_spacing=0.12)
  for i in range(input_arr.shape[0]):

    data=input_arr[i,:].copy()
    # Add traces
    fig.add_trace(go.Scatter(x=data.year, y=data,
                    mode='lines',name=model_list[i], opacity=.3),row=1, col=1)
    data=100*(input_arr[i,:]-input_arr[i,40:60].mean())/input_arr[i,40:60].mean()
    # Add traces
    fig.add_trace(go.Scatter(x=data.year, y=data,
                    mode='lines',showlegend=False, opacity=.3),row=2, col=1)

  data_std=sig_level*input_arr.std(dim='member').copy()/(n_ens_members-1)
  data=input_arr.mean(dim='member').copy()
  # Add traces
 
  fig.add_trace(go.Scatter(x=data.year, y=data.rolling(year=rolling_window,center=True).mean()+data_std.rolling(year=rolling_window,center=True).mean(),
                                     mode='lines',
                                     line=dict(color='black',width =0.1),showlegend=False),row=1,col=1)
  fig.add_trace(go.Scatter(x=data.year, y=data.rolling(year=rolling_window,center=True).mean(),
                         mode='lines',
                         line=dict(color='black'),
                         fill='tonexty',
                         name='Ensemble 20-year mean'),row=1,col=1)
  fig.add_trace(go.Scatter(x=data.year, y=data.rolling(year=rolling_window,center=True).mean()-data_std.rolling(year=rolling_window,center=True).mean(),
                         mode='lines',
                         line=dict(color='black', width =0.1),
                         fill='tonexty',showlegend=False),row=1,col=1)



  data_std=sig_level*100*(data_std)/data[40:60].mean()
 
  data=100*(input_arr[:,:]-input_arr[:,40:60].mean())/input_arr[:,40:60].mean()
  data=data.mean(dim='member').copy()
  # Add traces

  fig.add_trace(go.Scatter(x=data.year, y=data.rolling(year=rolling_window,center=True).mean()+data_std.rolling(year=rolling_window).mean(),
                                     mode='lines',
                                     line=dict(color='black',width =0.1),showlegend=False),row=2,col=1)
  fig.add_trace(go.Scatter(x=data.year, y=data.rolling(year=rolling_window,center=True).mean(),
                         mode='lines',
                         line=dict(color='black'),
                         fill='tonexty',showlegend=False),row=2,col=1)
  fig.add_trace(go.Scatter(x=data.year, y=data.rolling(year=rolling_window,center=True).mean()-data_std.rolling(year=rolling_window).mean(),
                         mode='lines',
                         line=dict(color='black', width =0.1),
                         fill='tonexty',showlegend=False),row=2,col=1)

  fig.update_yaxes(title_text=axis1, row=1, col=1)
  fig.update_yaxes(title_text='% change', row=2, col=1)
  fig.update_layout(autosize=False,width=1000,height=800,title={'text':title_2,'x': 0.5})
  fig.show(config=config)
 
def save_timeseries(input_arr,variable):


  data_std=sig_level*input_arr.std(dim='member').copy()/(n_ens_members-1)
  data=input_arr.mean(dim='member').copy()
  x=data.year
  y=data.rolling(year=rolling_window,center=True).mean()

  d = {'Year_i': x, variable: y,'std': data_std }
  pd_f = pd.DataFrame(data=d)
  pd_f['Year'] = pd.to_datetime(pd_f['Year_i'], format='%Y')
  pd_f = pd_f.set_index('Year')
  pd_f.drop(['Year_i'], axis=1, inplace=True)


  pd_f.to_csv('data.csv')
  files.download('data.csv')
  return pd_f

def to_360day_monthly(da):
    '''Takes a DataArray. Change the 
    calendar to 360_day and precision to monthly.'''
    val = da.copy()
    time1 = da.time.copy()
    for itime in range(val.sizes['time']):
        try: 
          bb = val.time.values[itime].timetuple()
        except:
          ts = (val.time.values[itime] - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')

          bb = datetime.utcfromtimestamp(ts).timetuple()
        #bb = val.time.values[itime].timetuple()   
        time1.values[itime] = cftime.Datetime360Day(bb[0],bb[1],16)

    # We rename the time dimension and coordinate to time360 to make it clear it isn't 
    # the original time coordinate.
    val = val.rename({'time':'time360'})
    time1 = time1.rename({'time':'time360'})
    val = val.assign_coords({'time360':time1})
    return val



### END DEFINE FUNCTIONS

In [7]:
# initialize gcfs token
print('Setting up database queries')
gcs = gcsfs.GCSFileSystem(token='anon')
zstore = df_hist.zstore.values[0]
# create a mutable-mapping-style interface to the store 
mapper = gcs.get_mapper(zstore)

# open data and select time slice 
ds = xr.open_zarr(mapper, consolidated=True)


Setting up database queries


In [8]:
# initialize gcfs token
print('Setting up database queries')
gcs = gcsfs.GCSFileSystem(token='anon')

# check that there are enough ensemble members for the specified analysis, otherwise update the specified number
if len(models) < n_ens_members:
    n_ens_members=len(models)
    print('Ensemble members updated to '+str(n_ens_members))
else:
    models=models[:n_ens_members]
    

# these two loops pull the past and future ensemble data from the cloud
for i in range(n_ens_members):

    print('Processing member: '+models[i]+' ...')
    df_hist = df.query("activity_id== " + activity_1 + " & source_id == "+"'"+models[i]+"'"+" & member_id == "+variant+" & table_id == "+resolution+" & variable_id == "+variable+" & experiment_id == "+scenario_1)

    # future ensemble query
    df_ssp = df.query("activity_id== " + activity_2 + "  & source_id == "+"'"+models[i]+"'"+"  & member_id == "+variant+"  & table_id == "+resolution+" & variable_id == "+variable+" & experiment_id == "+scenario_2)

    zstore = df_hist.zstore.values[0]

    # create a mutable-mapping-style interface to the store 
    mapper = gcs.get_mapper(zstore)

    # open data and select time slice 
    ds = xr.open_zarr(mapper, consolidated=True)

    ds=to_360day_monthly(ds)
  
    ens_hist=ds[variable[1:-1]].sel(time360=slice(date_1_start,date_1_end))

  

    zstore = df_ssp.zstore.values[0]

    # create a mutable-mapping-style interface to the store 
    mapper = gcs.get_mapper(zstore)

    # open data and select time slice 
    ds = xr.open_zarr(mapper, consolidated=True)
    ds=to_360day_monthly(ds)
    ens_fut=ds[variable[1:-1]]#.sel(time360=slice(date_2_start,date_2_end))


    # rename dimensions to standard names
    ens_hist=standardize_dims(ens_hist)
    ens_fut=standardize_dims(ens_fut)
    ens_fut=ens_fut.sel(lon=xc,lat=yc, method="nearest").squeeze()
    ens_hist=ens_hist.sel(lon=xc,lat=yc, method="nearest").squeeze()
    
    ts=xr.merge([ens_hist,ens_fut], compat='no_conflicts').to_array().squeeze()
    #time = pd.date_range("1951-01-01", freq="M", periods=1752)
    #ens=ens.reindex({"time": time})
   
    # get timeseries
    #ts=ens.sel(lon=xc,lat=yc, method="nearest").squeeze()
    try:
        datetimeindex = ts.indexes['time360'].to_datetimeindex()
    except:
        datetimeindex = ts.indexes['time360']
    ts['time360']=datetimeindex
    ts = xr.DataArray(data=ts, dims=["time360"], coords=[ts.time360])
    if i == 0:
        ens_ts=ts
    else:
        ens_ts=xr.concat([ens_ts,ts], 'member')
    
    print('Finished processing member: '+models[i])
ens_ts=ens_ts.squeeze()

Setting up database queries
Processing member: EC-Earth3 ...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:116: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:116: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future



Finished processing member: EC-Earth3
Processing member: IPSL-CM5A2-INCA ...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: RuntimeWarning:

Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.



Finished processing member: IPSL-CM5A2-INCA
Processing member: FGOALS-f3-L ...


...

Finished processing member: NorESM2-LM
Processing member: CMCC-ESM2 ...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: RuntimeWarning:

Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.



Finished processing member: CMCC-ESM2
Processing member: EC-Earth3-Veg-LR ...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:116: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:116: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future



Finished processing member: EC-Earth3-Veg-LR


In [9]:
# download data

%time ens_ts.load()

CPU times: user 422 µs, sys: 0 ns, total: 422 µs
Wall time: 425 µs


<xarray.DataArray 'stack-fce808d6c01f24e2b7d320dd33553a68' (member: 15, time360: 1799)>
array([[271.32568, 270.63208, 275.3268 , ..., 288.67746, 283.0457 ,
        282.0174 ],
       [268.06723, 268.97818, 270.66257, ..., 287.87958, 282.45068,
        279.0726 ],
       [275.01132, 270.22162, 276.7179 , ..., 288.82635, 281.54218,
        279.13666],
       ...,
       [271.6137 , 277.20682, 281.2579 , ..., 285.96332, 280.4746 ,
        278.1755 ],
       [276.9698 , 276.12262, 277.62292, ..., 287.19266, 280.39282,
        278.81665],
       [269.1349 , 275.07822, 277.39056, ..., 286.98956, 282.13297,
        276.5925 ]], dtype=float32)
Coordinates:
  * time360  (time360) datetime64[ns] 1951-01-16 1951-02-16 ... 2100-12-16
Dimensions without coordinates: member
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    comment:        near-surface (usually, 2 meter) air temperature
    history:        2019-09-25T21:18:16Z altered by CMOR: Treated scalar dime...
    long_name:      Near-Surface Air Temperature
    standard_name:  air_temperature
    units:          K

In [10]:
# check if outout dataArray is populated 
ens_ts

<xarray.DataArray 'stack-fce808d6c01f24e2b7d320dd33553a68' (member: 15, time360: 1799)>
array([[271.32568, 270.63208, 275.3268 , ..., 288.67746, 283.0457 ,
        282.0174 ],
       [268.06723, 268.97818, 270.66257, ..., 287.87958, 282.45068,
        279.0726 ],
       [275.01132, 270.22162, 276.7179 , ..., 288.82635, 281.54218,
        279.13666],
       ...,
       [271.6137 , 277.20682, 281.2579 , ..., 285.96332, 280.4746 ,
        278.1755 ],
       [276.9698 , 276.12262, 277.62292, ..., 287.19266, 280.39282,
        278.81665],
       [269.1349 , 275.07822, 277.39056, ..., 286.98956, 282.13297,
        276.5925 ]], dtype=float32)
Coordinates:
  * time360  (time360) datetime64[ns] 1951-01-16 1951-02-16 ... 2100-12-16
Dimensions without coordinates: member
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    comment:        near-surface (usually, 2 meter) air temperature
    history:        2019-09-25T21:18:16Z altered by CMOR: Treated scalar dime...
    long_name:      Near-Surface Air Temperature
    standard_name:  air_temperature
    units:          K

In [11]:
# plot and download data
e2=ens_ts[:,:].interpolate_na(dim='time360')-diff_fac
e2=e2.groupby('time360.year').mean(dim='time360')

# makes the plots
# here, you can edit the variable name, title, and units for the figure
plot_timeseries(e2,models[:],'Temperature','ssp370','C')

# this line downloads the data
data_frame= save_timeseries(e2,'Temperature')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# For temperature, this cell calculated change in available workimg hours
# The calculation is based on: Dunne et. al. 2013, Reductions in labour capacity from heat stress under climate warming, Nature Climate Change
# https://www.nature.com/articles/nclimate1827



e2=ens_ts.interpolate_na(dim='time360')
e2=e2.groupby('time360.year').mean(dim='time360')
#convert to wet bulb temperature
e3=e2.values.flatten()
[e_temp,Teq,epott]=WetBulb(e3-273.15,101325.0*np.ones(e3.shape),82.0*np.ones(e3.shape),1)
e_temp=e_temp*0.7+0.3*(e3-273.15)
# labour capicity 
LC=100-25*np.maximum(0,e_temp-25)**(2/3)
LC=LC.reshape(n_ens_members,150)
LC = xr.DataArray(LC, coords=[e2.member.values, e2.year.values], dims=["member", "year"])

# here, you can edit the variable name, title, and units for the figure
fig=plot_timeseries(LC,models,'Labour capacity','ssp370','percent')


# this line downloads the data
data_frame= save_timeseries(LC,'Labour Capacity')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>